# Prédiction des émissions de CO2 d'un véhicule en fonction de ces caractéristiques

- Import des libraries et des données

In [1]:
# libraries used for data exploration
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import skim
from scipy.stats import kurtosis, skew
import pingouin as pg

# librairies used for machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor, export_text

# utils
import gc
from scipy.stats import randint

import warnings
warnings.filterwarnings('ignore')

In [2]:
def distribution_analysis(dataframe, variable, dataframe_and_variable):
    sns.histplot(
    data = dataframe,
    x = variable,
    kde=True
    )

    plt.title(f"Distribution de la variable {variable}")
    plt.show()

    skewness = skew(dataframe_and_variable)
    kurtosis_var = kurtosis(dataframe_and_variable)
    normality = pg.normality(
        dataframe_and_variable,
        method="normaltest"
        )

    print(f"Skewness : {skewness}  - Kurtosis : {kurtosis_var}\nTest de normalité de la distribution\n{normality}")

def numerical_analysis(dataframe, variable):
    sns.boxplot(
    x=variable,
    data=dataframe
    )
    plt.title(f'Boîte à moustaches de la variable {variable}')
    plt.show()

    print(dataframe[variable].describe())

def categorical_analysis(dataframe, variable):
    sns.countplot(
        x = variable,
        data = dataframe,
        stat="percent"
    )
    plt.title(f"Répartition des véhicules en fontion de la variable : {col}")
    plt.ylabel("Pourcentage (%)")
    plt.xticks(rotation=90)
    plt.show()

def linear_regression_analysis(dataframe, variable, target):
    sns.lmplot(
        data=dataframe,
        x=target,
        y=variable
        )
    plt.title(f"Interaction entre {target} et {variable}")
    plt.show()
    lm = pg.linear_regression(dataframe[target], dataframe[variable])
    print(lm.round(2))

def categorical_relation(dataframe, variable, target):
    sns.boxplot(
        data=dataframe,
        x=variable,
        y=target
        )
    plt.title(f"Interaction entre {target} et {variable}")
    plt.xticks(rotation=90)
    plt.show()

    aov = pg.anova(
        dv=target,
        between=variable,
        data=dataframe,
        detailed=True
        )
    print(aov.round(2))

In [3]:
conn = sqlite3.connect('../datasets/ademe_car_labeling.db')

In [15]:
df = pd.read_sql("SELECT * FROM car_information", con=conn)

In [16]:
df.head()

,index,Marque,Modèle,Energie,Carrosserie,Cylindrée,Gamme,Puissance fiscale,Puissance maximale,Puissance nominale électrique,...,Bonus-Malus,Conso basse vitesse,Conso moyenne vitesse,Conso haute vitesse,Conso T-haute vitesse,Conso vitesse mixte,Conso elec,Autonomie elec,Autonomie elec urbain,Emission CO2
0,0,RENAULT,KANGOO,ESSENCE,COMBISPACE,1332,INFERIEURE,7,96.0,NaN,...,Malus,8.7405,6.7880,6.188,7.9175,7.2635,NaN,NaN,NaN,167.272
1,1,MAZDA,MX-30,ELECTRIC,TS TERRAINS/CHEMINS,0,INFERIEURE,6,NaN,80.9,...,Bonus 6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,DS,7 CROSSBACK,ELEC+ESSENC HR,TS TERRAINS/CHEMINS,1598,MOYENNE SUPERIEURE,10,133.0,30.0,...,Neutre 0,NaN,NaN,NaN,NaN,1.5340,162.5,58.5,67.5,NaN
3,3,RENAULT,AUSTRAL,ESS+ELEC HNR,TS TERRAINS/CHEMINS,1199,MOYENNE SUPERIEURE,7,96.0,0.0,...,Neutre 0,6.5805,5.0200,4.834,5.9555,5.5025,NaN,NaN,NaN,126.073
4,4,B.M.W.,218,ESSENCE,MONOSPACE COMPACT,1499,MOYENNE INFERIEURE,7,100.0,NaN,...,Malus,8.9420,6.6295,5.637,6.4505,6.5680,NaN,NaN,NaN,155.826


In [17]:
# create list to separe categorical & numerical variables
# remove the target variables
categorical = list(df.dtypes[df.dtypes == 'object'].index)
numerical = [i for i in list(df.columns) if i not in categorical]
numerical.pop(-1)

'Emission CO2'

In [18]:
for col in categorical:
    df[col] = df[col].str.lower().str.replace(" ", "_")

In [19]:
df.head()

,index,Marque,Modèle,Energie,Carrosserie,Cylindrée,Gamme,Puissance fiscale,Puissance maximale,Puissance nominale électrique,...,Bonus-Malus,Conso basse vitesse,Conso moyenne vitesse,Conso haute vitesse,Conso T-haute vitesse,Conso vitesse mixte,Conso elec,Autonomie elec,Autonomie elec urbain,Emission CO2
0,0,renault,kangoo,essence,combispace,1332,inferieure,7,96.0,NaN,...,malus,8.7405,6.7880,6.188,7.9175,7.2635,NaN,NaN,NaN,167.272
1,1,mazda,mx-30,electric,ts_terrains/chemins,0,inferieure,6,NaN,80.9,...,bonus_6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ds,7_crossback,elec+essenc_hr,ts_terrains/chemins,1598,moyenne_superieure,10,133.0,30.0,...,neutre_0,NaN,NaN,NaN,NaN,1.5340,162.5,58.5,67.5,NaN
3,3,renault,austral,ess+elec_hnr,ts_terrains/chemins,1199,moyenne_superieure,7,96.0,0.0,...,neutre_0,6.5805,5.0200,4.834,5.9555,5.5025,NaN,NaN,NaN,126.073
4,4,b.m.w.,218,essence,monospace_compact,1499,moyenne_inferieure,7,100.0,NaN,...,malus,8.9420,6.6295,5.637,6.4505,6.5680,NaN,NaN,NaN,155.826


## 1) Préparation des données

In [20]:
df.shape

(7679, 24)

In [21]:
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 7679   │ │ float64     │ 12    │                                                          │
│ │ Number of columns │ 24     │ │ string      │ 7     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 5     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name             ┃ NA    ┃ NA %   ┃ mean   ┃ sd     ┃ p0    ┃ p25   ┃ p50   ┃ p75   ┃ p100 ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ index                   │     0 │      0 │   3800 │   2200 │     0 │  1900 │  3800 │  5800 │ 7700 │ ▇▇▇▇▇▇ │  │
│ │ Cylindrée               │     0 │      0 │   1700 │    680 │     0 │  1400 │  1600 │  2000 │ 6700 │  ▂▇▁   │  │
│ │ Puissance fiscale       │     0 │      0 │     10 │    7.7 │     1 │     6 │     8 │    10 │   91 │   ▇▁   │  │
│ │ Puissance maximale      │   269 │    3.5 │    130 │     66 │    48 │    92 │   110 │   140 │  610 │  ▇▂▁   │  │
│ │ Puissance nominale      │  4925 │  64.14 │     25 │     32 │     0 │     0 │    10 │    48 │  200 │  ▇▂▁   │  │
│ │ électrique              │       │        │        │        │       │       │       │       │      │        │  │
│ │ Poids à vide            │     0 │      0 │   1600 │    340 │   840 │  1300 │  1500 │  1800 │ 2700 │ ▂▇▆▃▁  │  │
│ │ Rapport poids-puissance │     0 │      0 │  0.079 │  0.035 │     0 │  0.06 │  0.07 │  0.09 │ 0.37 │  ▃▇▁   │  │
│ │ Nombre rapports         │     0 │      0 │    6.7 │    1.9 │     0 │     6 │     7 │     8 │    9 │ ▁   ▇▆ │  │
│ │ Conso basse vitesse     │   989 │  12.88 │      8 │    2.2 │   3.2 │   6.7 │   7.8 │   8.9 │   30 │   ▇▇   │  │
│ │ Conso moyenne vitesse   │   989 │  12.88 │    6.1 │    1.3 │   3.3 │   5.2 │   5.9 │   6.7 │   18 │  ▆▇▁   │  │
│ │ Conso haute vitesse     │   989 │  12.88 │    5.4 │      1 │   3.7 │   4.7 │   5.3 │   5.8 │   14 │  ▇▆▁   │  │
│ │ Conso T-haute vitesse   │   989 │  12.88 │    6.5 │    1.1 │   4.2 │   5.7 │   6.3 │     7 │   14 │  ▅▇▂   │  │
│ │ Conso vitesse mixte     │   395 │   5.14 │    5.9 │    1.7 │  0.56 │   5.2 │     6 │   6.8 │   17 │  ▁▆▇   │  │
│ │ Conso elec              │  6861 │  89.35 │    200 │     42 │   130 │   160 │   190 │   240 │  290 │ ▇▇▅▃▆▂ │  │
│ │ Autonomie elec          │  6861 │  89.35 │    140 │    140 │    40 │    50 │    64 │   120 │  680 │  ▇ ▁▁  │  │
│ │ Autonomie elec urbain   │  6861 │  89.35 │    170 │    180 │    41 │    55 │    74 │   170 │  740 │ ▇  ▁▁  │  │
│ │ Emission CO2            │   989 │  12.88 │    150 │     31 │    93 │   140 │   150 │   160 │  410 │  ▇▇▁   │  │
│ └─────────────────────────┴───────┴────────┴────────┴────────┴───────┴───────┴───────┴───────┴──────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                  ┃ NA     ┃ NA %       ┃

Les véhicules électriques n'émettent pas de CO2, ils sont enlevés du jeu de données pour la suite de l'analyse ainsi que pour la construction du modèle de prédiction. 

In [22]:
# remove electric vehicles
df = df[df["Energie"] != 'electric']

In [23]:
df.describe()

,index,Cylindrée,Puissance fiscale,Puissance maximale,Puissance nominale électrique,Poids à vide,Rapport poids-puissance,Nombre rapports,Conso basse vitesse,Conso moyenne vitesse,Conso haute vitesse,Conso T-haute vitesse,Conso vitesse mixte,Conso elec,Autonomie elec,Autonomie elec urbain,Emission CO2
count,7409.000000,7409.000000,7409.000000,7409.000000,2484.000000,7409.000000,7409.000000,7409.000000,6690.000000,6690.000000,6690.000000,6690.000000,7284.000000,594.000000,594.000000,594.000000,6690.000000
mean,3844.402483,1796.202322,10.229856,130.021055,20.157009,1555.344986,0.082061,6.928195,8.018916,6.075571,5.407636,6.464161,5.897295,199.928636,61.744613,68.052020,154.651505
std,2214.697856,609.909070,7.806961,66.386992,27.228414,333.232021,0.031618,1.553918,2.238887,1.254111,1.027638,1.135738,1.719087,40.243956,18.477725,19.951523,30.739784
min,0.000000,875.000000,4.000000,48.000000,0.000000,840.000000,0.040000,0.000000,3.219000,3.327000,3.726000,4.238500,0.562000,133.500000,39.700000,41.350000,92.947000
25%,1926.000000,1469.000000,6.000000,91.900000,0.000000,1320.000000,0.060000,6.000000,6.663000,5.213500,4.709000,5.655000,5.243500,162.500000,48.000000,51.750000,135.540000
50%,3843.000000,1950.000000,8.000000,110.000000,8.000000,1490.000000,0.080000,7.000000,7.765000,5.912500,5.267500,6.296000,5.987000,192.850000,56.000000,61.000000,147.635000
75%,5760.000000,1995.000000,10.000000,140.000000,30.000000,1755.000000,0.090000,8.000000,8.862500,6.675500,5.840500,7.025000,6.752500,241.770000,74.000000,87.000000,164.659250
max,7678.000000,6749.000000,91.000000,607.000000,139.000000,2735.000000,0.370000,9.000000,29.914500,17.541500,13.632000,13.947000,16.521500,288.500000,112.000000,116.500000,411.010000


In [24]:
df.columns

Index(['index', 'Marque', 'Modèle', 'Energie', 'Carrosserie', 'Cylindrée',
       'Gamme', 'Puissance fiscale', 'Puissance maximale',
       'Puissance nominale électrique', 'Poids à vide',
       'Rapport poids-puissance', 'Type de boite', 'Nombre rapports',
       'Bonus-Malus', 'Conso basse vitesse', 'Conso moyenne vitesse',
       'Conso haute vitesse', 'Conso T-haute vitesse', 'Conso vitesse mixte',
       'Conso elec', 'Autonomie elec', 'Autonomie elec urbain',
       'Emission CO2'],
      dtype='object')

On remarque que les variables en lien avec la consommation électrique des véhicules présentent un grand nombre de variables manquantes. Cela s'explique par le fait que les véhicules thermiques classiques ne vont pas consommer d'électricité. On va donc remplacer ces valeurs par 0.

In [ ]:
# fill variables about electric consumption

electrical_variables  = [
    "Autonomie elec",
    "Puissance nominale électrique",
    "Conso elec",
    "Autonomie elec urbain"
    ]

for col in electrical_variables:
    df[col] = df[col].fillna(0)

Concernant les autres variables numériques avec des valeurs manquantes ainsi que la variable cible, on complète les valeurs manquantes par la valeur médiane. 

In [ ]:
# fill other numerical variables
for c in numerical:
    df[c] = df[c].fillna(df[c].median())

In [ ]:
# fill target variable
df["Emission CO2"] = df["Emission CO2"].fillna(df["Emission CO2"].median())

In [ ]:
skim(df)

Les variables manquantes ont été supprimées, on peut donc passer à l'analyse exploratoire des doonées.

## 2) Analyse exploratoire des données

### a) Analyses univariées

#### Analyse de la variable cible

- Distribution de la variable cible : Emission de CO2

In [ ]:
distribution_analysis(df, "Emission CO2", df["Emission CO2"])

Le coefficient d'asymétrie (skewness) montre une asymétrie de la distribution et le kurtosis indique la présence d'un pic important dans la distribution des émissions de CO2. Enfin le test de normalité montre que la distribution ne suit pas la loi normale.

Il faudra donc effectuer une transformation logarithmique pour éviter que la forme de la distribution perturbe les modèles linéaires qui pourraient être utilisés dans notre recherche de modèle.

- Indices de tendance centrale et de dispersion : Emission de CO2

In [ ]:
numerical_analysis(df, "Emission CO2")

Les émissions de CO2 sont comprises entre 92.947 g et 411.01 g. La moyenne est de 153.97g et la médiane de 147.63g. L'écart-type est de 29.28g et l'intervalle interquartile de 25.652g. 

In [ ]:
# log transformation
df["Emission CO2"] = np.log1p(df["Emission CO2"])

#### Analyse des variables catégorielles

In [ ]:
for col in ['Energie', 'Carrosserie', 'Gamme', 'Bonus-Malus', "Type de boite"]:
    categorical_analysis(df, col)

In [ ]:
plt.figure(figsize=(15,15))

sns.countplot(
    y="Marque",
    data=df,
    stat="percent"
)
plt.ylabel("Pourcentage (%)")
plt.title(f"Répartition des véhicules selon les marques")
plt.show()

In [ ]:
for col in numerical:
    distribution_analysis(df, col, df[col])

In [ ]:
for col in numerical:
    numerical_analysis(df, col)

### 2) Analyse bivariée

#### Variables quantitatives

In [ ]:
numerical.append("Emission CO2")

- Matrice de corrélation

In [ ]:
corr_matrix = df[numerical].corr()
mask = np.triu(corr_matrix)

plt.figure(figsize=(15,15))
sns.heatmap(corr_matrix,
            mask=mask,
            linewidths=.5,
            annot=True,
            cbar=True,
            square=True)
plt.title("Matrice de corrélation entre les différentes variables quantitatives.")
plt.show()

In [ ]:
# removing variables with poor correlation coefficient
del df["Puissance nominale électrique"]
del df["Conso elec"]
del df["Autonomie elec"]
del df["Autonomie elec urbain"]

In [ ]:
variables_to_remove = ["Puissance nominale électrique", "Conso elec", "Autonomie elec", "Autonomie elec urbain", "Emission CO2"]
numerical = [x for x in numerical if x not in variables_to_remove]

- Régression linéaire entre émission de CO2 et les variables quantitatives

In [ ]:
for col in numerical:
    linear_regression_analysis(df, col, "Emission CO2")

### Variables catégorielles

In [ ]:
for col in ['Energie', 'Carrosserie', 'Gamme', 'Bonus-Malus', "Type de boite"]:
    categorical_relation(df, col, "Emission CO2")

## 3) Préparation des données pour modélisation

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train["Emission CO2"].values
y_test = df_test["Emission CO2"].values

In [ ]:
del df_train['Emission CO2']
del df_test['Emission CO2']

In [ ]:
numerical = [i for i in list(df.columns) if i not in categorical]
numerical.pop(-1)

In [ ]:
train_dicts = df_train.to_dict(orient='records')
test_dicts = df_test.to_dict(orient='records')

In [ ]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_test = dv.transform(test_dicts)

### 4) Choix du modèle

#### Baseline

In [ ]:
dummy = DummyRegressor(strategy="median")
dummy.fit(X_train, y_train)

In [ ]:
y_pred = dummy.predict(X_test)

In [ ]:
rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 5)
mape = round(mean_absolute_percentage_error(y_test, y_pred), 5)
print(f"rmse : {rmse} & mape : {mape}")

In [ ]:
del y_pred
del rmse
del mape
gc.collect()

#### Random Forest Regressor

In [ ]:
rf = RandomForestRegressor(random_state=42,
                           n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 5)
mape = round(mean_absolute_percentage_error(y_test, y_pred), 5)
print(f"rmse : {rmse} & mape : {mape}")

In [ ]:
param_distribs = {
    "n_estimators" : [int(x) for x in np.linspace(50, 500, 10)],
    "max_depth" : [int(x) for x in np.linspace(5, 16, 10)],
    "min_samples_leaf" : randint(2, 10),
    "min_samples_split" : randint(2, 10),
    "bootstrap" : [True, False]

}

rf_rnd_search = RandomizedSearchCV(rf,
                                   param_distributions=param_distribs,
                                   n_iter=100,
                                   cv=5,
                                   scoring="neg_mean_squared_error",
                                   random_state=42)
rf_rnd_search.fit(X_train, y_train)

In [ ]:
rf_rnd_search.best_params_

In [ ]:
params = [{
    "n_estimators" : [120, 130, 140, 150],
    "max_depth" : [11, 12, 13],
    "min_samples_leaf" : [2, 3],
    "min_samples_split" : [2, 3],
    "bootstrap" : [True]
}]

rf_grid_search = GridSearchCV(rf,
                            params,
                            cv=5,
                            scoring="neg_mean_squared_error")
rf_grid_search.fit(X_train,
                   y_train)

In [ ]:
rf_grid_search.best_params_

In [ ]:
rf = rf_grid_search.best_estimator_

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 5)
mape = round(mean_absolute_percentage_error(y_test, y_pred), 5)
print(f"rmse : {rmse} & mape : {mape}")

In [ ]:
df_importances = pd.DataFrame()
df_importances['feature'] = dv.feature_names_
df_importances['importance'] = rf.feature_importances_
df_importances.sort_values(by="importance", ascending=False).head(10)